# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')

# 실제시험 x
s_ans = pd.read_csv('test_prob_ans.csv', index_col=['id'])['failure']
df_train.shape, df_test.shape

((21458, 25), (5112, 24))

In [3]:
df_train['na_1'] = df_train['measurement_3'].isna()
df_train['na_2'] = df_train['measurement_5'].isna()

df_test['na_1'] = df_test['measurement_3'].isna()
df_test['na_2'] = df_test['measurement_5'].isna()

In [4]:
# 방법 1: train과 test의 product_code의 구성을 고려한 최적 방법

from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer # , random_state=123
imp = IterativeImputer(
    estimator=LinearRegression(fit_intercept=True), random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))

In [5]:
# 방법 2 : train / test를 통합하여 처리해봅니다.

from sklearn.experimental import enable_iterative_imputer# 구문을 사용하여 실험 단계인 모듈을 활성화하고, 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
# train에 등장하지 않은 수준이 있습니다, test를 포함하여 결측처리 모델을 만듭니다.
s_imp = pd.concat([
        df_train[X_imp + ['product_code']],
        df_test[X_imp + ['product_code']]
], axis=0).groupby('product_code')\
.apply(
    lambda x: IterativeImputer(estimator=LinearRegression(),random_state=123).fit(x[X_imp])
)
# train에 적용합니다.
df_train[X_imp] = df_train[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
# test에 적용합니다.
df_test[X_imp] = df_test[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
# train에 등장하지 않은 범주를 처리하기 위해 합치니다.
df_mean = pd.concat([
            df_train[['product_code'] + X_mean],
            df_test[['product_code'] + X_mean]
        ]).groupby('product_code')[X_mean].agg('mean')

df_train[X_mean] = df_train.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))
df_test[X_mean] = df_test.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))

In [9]:
m = pd.concat([df_train['loading'], df_test['loading']]).mean()
df_train['loading'] = df_train['loading'].fillna(m)
df_test['loading'] = df_test['loading'].fillna(m)
X_all = df_test.columns.tolist()

In [10]:
df_train.isna().sum().pipe(
    lambda x: x.loc[x > 0]
)

Series([], dtype: int64)

## Step1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Day1에서 했던 Holdout 검증보다 더 안정적인 평가 결과를 얻을 수 있는 교차 검증을 해봅니다.

### GroupKFold에 대해 알아봅니다.

Test의 product_code는 train에서 등장하지 않았습니다.

이를 기반으로 검증 방법을 정해보면, 

product_code에 대해서 검증셋에서는 검증 학습셋에서 등장하지 않는 product_code가 되도록 구성을 하면, 

Test의 현상을 반영하는 검증 루틴을 구성할 수 있습니다.

이에 활용할 수 있는 검증법이 바로 GroupKFold 입니다.

In [11]:
from sklearn.model_selection import GroupKFold

gcv = GroupKFold(4)
for train_idx, valid_idx in gcv.split(df_train[X_all], df_train['failure'], groups=df_train['product_code']):
    df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    print(df_cv_train['product_code'].unique(), df_valid['product_code'].unique())

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'] ),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], scoring='roc_auc', cv=gcv, groups = df_train['product_code'],
    return_train_score=True
)

{'fit_time': array([0.03427458, 0.03095746, 0.02125573, 0.03176475]),
 'score_time': array([0.00419188, 0.01000667, 0.01005673, 0.00568748]),
 'test_score': array([0.58821746, 0.58491734, 0.58894014, 0.59540058]),
 'train_score': array([0.59262252, 0.59350804, 0.59192438, 0.58956303])}

In [13]:
# train / test 에서의 product_code가 등장하는 양상에 맞춰 검증 루틴을 구성해봅니다. (Step 1)
s_hist = list()

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

gcv = GroupKFold(4)
def eval_model(model_name, model):
    """
        모델의 인스턴스를 받아 검증 결과(AUC) 를 구합니다.
        1. GroupKFold 검증 - df_train, groups - product_code
        2. df_valid에 대한 예측결과에 대한 AUC를 구합니다.
        3. 주어진 모델명으로 평가 결과를 저장합니다. Format: Valid: {:.5f}±{:.5f}, Train: {:.5f}±{:.5f}
        4. 가장 최근의 수행결과를 보여 주어 선택하는데 활용하도록 합니다.
    Parameters:
        model_name: str, 모델의 이름,
        model: sklearn object, 모델 인스턴스
    """
    

# 모델 선택 루틴입니다. (Step 3)
def select_model(model):
    """
        1. 전체 학습데이터(df_train)로 학습을 합니다. 
        2. df_test에 대한 예측을 합니다.
        3. 예측 결과를 출력양식(id, failure)에 맞춰 csv파일을 만듭니다.
        4. 자가 채점을 위한 예측 결과를 반환합니다.
    Parameters: 
        model: sklearn object, 모델 인스턴스
    Returns: 1차원 np.ndarray
        예측 결과
    """
    return np.zeros(len(df_test))